<a href="https://colab.research.google.com/github/seunghuilee91/Deeplearning/blob/master/DBSCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append("/content/gdrive/My Drive/Colab Notebooks")

# DBSCAN
Density-based spatial clustering of applications with noise

---


- Two algorithm parameters 
 - ε (eps) 
 - the minimum number of points required to form a dense region(minPts).
- Catures arbitrary shape of clusters and does not require the number of clusters in advance
- Find dense regions and expands them in order to form clusters
<br>
<br>

클러스터링 알고리즘은 크게 **중심 기반 (center-based) 알고리즘**과 **밀도 기반 (density-based) 알고리즘**으로 나눌 수 있다. 중심 기반 알고리즘의 가장 대표적인 것으로는 k-means clustering이 있으며, 밀도 기반 알고리즘에는 DBSCAN이 있다.
<br>
<br>

<img src="https://t1.daumcdn.net/cfile/tistory/99D207335B96763F18?download">

## Advantages & Disadvantages
- **DBSCAN does not require one to specify the number of clusters in the data a priori, as opposed to k-means.**

 → 알고리즘이 자동으로 클러스터의 수를 찾는다.

- **DBSCAN can find arbitrarily shaped clusters. It can even find a cluster completely surrounded by (but not connected to) a different cluster. Due to the MinPts parameter, the so-called single-link effect (different clusters being connected by a thin line of points) is reduced.**

 → Gaussian 분포가 아닌, 불특정한 분포를 따르는 데이터에 대해서는 밀도 기반 클러스터링 알고리즘을 활용하는 것이 적절함, 원 모양의 클러스터뿐만 아니라, 불특정한 모양의 클러스터도 찾을 수 있다.


- **DBSCAN has a notion of noise, and is robust to outliers**.

 → 클러스터링을 수행하는 동시에 노이즈 데이터도 분류할 수 있기 때문에 outlier에 의해 클러스터링 성능이 하락하는 현상을 완화할 수 있다.

- **DBSCAN requires just two parameters and is mostly insensitive to the ordering of the points in the database. (However, points sitting on the edge of two different clusters might swap cluster membership if the ordering of the points is changed, and the cluster assignment is unique only up to isomorphism.)**

 → 데이터가 입력되는 순서에 따라 클러스터링 결과가 변한다.

- **DBSCAN is designed for use with databases that can accelerate region queries, e.g. using an R tree.**

- **The parameters minPts and ε can be set by a domain expert, if the data is well understood, however If the data and scale are not well understood, choosing a meaningful distance threshold ε can be difficult.**

 → 데이터의 특성을 모를 경우에는 알고리즘의 적절한 hyper-parameter를 설정하기가 어렵다.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/DBSCAN-density-data.svg/330px-DBSCAN-density-data.svg.png">

## Implemention 
DBSCAN은 각각의 데이터들에 대해 이웃한 데이터와의 밀도를 계산하면서 불특정한 모양의 클러스터를 생성한다. 따라서, DBSCAN을 정의하기 위해서는 이웃한 데이터와 클러스터에 대한 정의가 필요하다. 이를 위해 DBSCAN의 hyper-parameter로 주어지는 주변 거리 ϵ과 최소 이웃 데이터의 수 nc를 기반으로 아래와 같은 개념들을 정의한다.

> - 이웃 데이터: 어떠한 데이터 x와의 거리가 ϵ 보다 가깝거나 같은 데이터를 x의 이웃 데이터라고 한다.
> - 핵심 데이터: 이웃 데이터의 수가 nc보다 많거나 같은 데이터를 핵심 데이터라고 한다.
> - 외곽 데이터: 핵심 데이터는 아니지만, 어떠한 핵심 데이터의 이웃 데이터인 것을 말한다.
> - 노이즈 데이터: 핵심 데이터도 아니며, 외곽 데이터도 아닌 것을 말한다. 즉, 거리 ϵ 이내에 nc개 미만의 데이터가 있으며, 그 데이터들 모두 핵심 데이터가 아닐 경우를 노이즈 데이터라고 한다.


<IMG SRC="https://t1.daumcdn.net/cfile/tistory/99CC563359E057BA25">

정리해서 이야기 하면, 점을 중심으로 epsilon 반경내에 minPts 이상수의 점이 있으면 그 점을 중심으로 군집이 되고 그 점을 core point라고 한다. Core point 가 서로 다른 core point의 군집의 일부가 되면 그 군집을 서로 연결되어 있다고 하고 하나의 군집으로 연결을 한다. 

 군집에는 속하지만, 스스로 core point가 안되는 점을 border point라고 하고, 주로 클러스터의 외곽을 이루는 점이 된다. 

 

[예제코드]

In [ ]:
import pandas as pd
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
iris

In [ ]:

labels = pd.DataFrame(iris.target)
labels.columns=['labels']
data = pd.DataFrame(iris.data)
data.columns=['Sepal length','Sepal width','Petal length','Petal width']
data = pd.concat([data,labels],axis=1)

data.head()



In [ ]:
# IRIS 데이타를 DataFrame으로 로딩 한 다음, 학습에 사용할 피쳐를 다음과 같이 feature 변수에 저장한다.

feature = data[ ['Sepal length','Sepal width','Petal length','Petal width']]
feature.head()



In [ ]:
# 다음은 모델을 선언하고, 데이타를 넣어서 학습을 시킨다. 

from sklearn.cluster import DBSCAN
import matplotlib.pyplot  as plt
import seaborn as sns


# create model and prediction
model = DBSCAN(min_samples=6)
predict = pd.DataFrame(model.fit_predict(feature))
predict.columns=['predict']

# concatenate labels to df as a new column
r = pd.concat([feature,predict],axis=1)



In [ ]:
# 다음은 모델을 선언하고, 데이타를 넣어서 학습을 시킨다. 
# 학습이 끝난 결과를 다음과 같이 3차원 그래프로 시각화 해보자. 아래 시각화는 3차원인데, 학습은 4차원으로 하였다. 그래서 다소 오류가 있어 보일 수 있다. 다차원 데이타를 시각화 하기위해서는 PCA나 t-SNE와 같은 차원 감소 (dimensional reduction) 기법을 사용해야 하는데,  이는 다음 글에서 다루도록한다.


from mpl_toolkits.mplot3d import Axes3D
# scatter plot
fig = plt.figure( figsize=(6,6))
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
ax.scatter(r['Sepal length'],r['Sepal width'],r['Petal length'],c=r['predict'],alpha=0.5)
ax.set_xlabel('Sepal lenth')
ax.set_ylabel('Sepal width')
ax.set_zlabel('Petal length')
plt.show()

In [ ]:
ct = pd.crosstab(data['labels'],r['predict'])
print (ct)